In [1]:
pip install pyodbc pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
import pyodbc
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


## RECALL

In [39]:
conn1 = pyodbc.connect(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=desktop-h3rbqbo;"
    "DATABASE=Recall_NHTSA;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

# SQL Query to fetch all data
query1 = "SELECT * FROM dbo.NHTSA_Recall;"

# Load table data into Pandas DataFrame
recall = pd.read_sql(query1, conn1)


# Close connection
conn1.close()

C:\Users\shanm\AppData\Local\Temp\ipykernel_18572\1636387689.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  recall = pd.read_sql(query1, conn1)


In [27]:
recall.head()

,RECORD_ID,CAMPNO,MAKETXT,MODELTXT,YEARTXT,MFGCAMPNO,COMPNAME,MFGNAME,BGMAN,ENDMAN,...,DATEA,RPNO,FMVSS,DESC_DEFECT,CONEQUENCE_DEFECT,CORRECTIVE_ACTION,NOTES,MFR_COMP_NAME,MFR_COMP_DESC,MFR_COMP_PTNO
0,1,02V288000,FORD,FOCUS,2000,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,1999-07-19,2001-05-31,...,2002-11-06,None,None,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,None,None,None
1,2,02V288000,FORD,FOCUS,2001,02S41,ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES,FORD MOTOR COMPANY,1999-07-19,2001-05-31,...,2002-11-06,None,None,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...,None,None,None
2,3,02V236000,JAYCO,FT EAGLE 10 SG,2003,None,EQUIPMENT:OTHER:LABELS,"JAYCO, INC.",2002-07-30,2002-08-13,...,2002-09-12,None,None,"ON CERTAIN FOLDING TENT CAMPERS, THE FEDERAL C...","IF THE TIRES WERE INFLATED TO 80 PSI, THEY COU...",OWNERS WILL BE MAILED CORRECT LABELS FOR INSTA...,"ALSO, CUSTOMERS CAN CONTACT THE NATIONAL HIGHW...",None,None,None
3,4,02V237000,HOLIDAY RAMBLER,ENDEAVOR,2000,None,STRUCTURE,MONACO COACH CORP.,None,None,...,2002-09-12,None,None,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,None,None,None
4,5,02V237000,HOLIDAY RAMBLER,ENDEAVOR,1999,None,STRUCTURE,MONACO COACH CORP.,None,None,...,2002-09-12,None,None,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...,None,None,None


In [28]:
recall.shape

(583658, 26)

In [40]:
#dropping all rows which is initiated by MFR. We are considering only customer initiated complaints
recall = recall[recall['INFLUENCED_BY'] != 'MFR']

In [41]:
#dropping all duplicates values in the primary keys. not performing any aggregation and imputation since we only need indicator for reacll
RCL_unique = recall.drop_duplicates(subset=['MAKETXT', 'MODELTXT', 'YEARTXT', 'COMPNAME'])

In [42]:
RCL_unique.shape

(35599, 26)

---

## INVESTIGATIONS

In [43]:
conn2 = pyodbc.connect(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=desktop-h3rbqbo;"
    "DATABASE=Investigation;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

# SQL Query to fetch all data
query2 = "SELECT * FROM dbo.Investigations;"

# Load table data into Pandas DataFrame
investigations = pd.read_sql(query2, conn2)


# Close connection
conn2.close()

C:\Users\shanm\AppData\Local\Temp\ipykernel_18572\2752787240.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  investigations = pd.read_sql(query2, conn2)


In [44]:
investigations.head()

,NHTSA_ACTION_NUMBER,MAKE,MODEL,YEAR,COMPNAME,MFR_NAME,ODATE,CDATE,CAMPNO,SUBJECT,SUMMARY
0,AQ08001,PACE AMERICAN,TRAILER,2003,WHEELS,"PACE AMERICAN, INC.",2008-06-18,2008-10-29,None,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
1,AQ08001,PACE AMERICAN,TRAILER,2004,WHEELS,"PACE AMERICAN, INC.",2008-06-18,2008-10-29,None,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
2,AQ08001,PACE AMERICAN,TRAILER,2005,WHEELS,"PACE AMERICAN, INC.",2008-06-18,2008-10-29,None,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
3,AQ08001,PACE AMERICAN,TRAILER,2006,WHEELS,"PACE AMERICAN, INC.",2008-06-18,2008-10-29,None,PACE AMERICAN 573 RETRACTION,"BY LETTER DATED NOVEMBER 8, 2007, PACE AMERICA..."
4,AQ09001,APEXCONE,APX9004,9999,EXTERIOR LIGHTING,CARFILO,2009-03-26,2009-07-06,05E069000,HID REPLACEMENT KIT RECALL CAMPAIGNS,RMD IDENTIFIED SEVERAL HID REPLACEMENT LIGHTIN...


In [7]:
#check
investigations.groupby(['MAKE', 'MODEL', 'YEAR', 'COMPNAME'])['NHTSA_ACTION_NUMBER'].nunique().reset_index().sort_values(by = 'NHTSA_ACTION_NUMBER', ascending = False)		

,MAKE,MODEL,YEAR,COMPNAME,NHTSA_ACTION_NUMBER
12179,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,47
12184,FORD,E350,1984,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,46
12188,FORD,E350,1985,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,46
12193,FORD,E350,1986,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,46
12198,FORD,E350,1987,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,46
...,...,...,...,...,...
14570,FORD,FUSION HYBRID,2015,ELECTRICAL SYSTEM:PROPULSION SYSTEM,1
14569,FORD,FUSION HYBRID,2015,AIR BAGS:FRONTAL:PASSENGER SIDE:INFLATOR MODULE,1
14568,FORD,FUSION HYBRID,2015,AIR BAGS:FRONTAL:DRIVER SIDE:INFLATOR MODULE,1
14567,FORD,FUSION HYBRID,2014,STRUCTURE:BODY:DOOR,1


In [11]:
#check
investigations[(investigations['MAKE'] == 'FORD') & (investigations['MODEL'] == 'E350') & (investigations['YEAR'] == '1983') & (investigations['COMPNAME'] == 'VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM')]

,NHTSA_ACTION_NUMBER,MAKE,MODEL,YEAR,COMPNAME,MFR_NAME,ODATE,CDATE,CAMPNO,SUBJECT,SUMMARY
2299,PE89047,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-17,1989-03-15,None,UNDERHOOD FIRES,===
27041,PE89048,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-17,1989-07-21,None,UNDERHOOD FIRES,===
27046,PE89049,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-17,1989-06-15,None,UNDERHOOD FIRES,SUMMARY:AMERICAN MODULAR BODY WENT BANKRUPT AN...
28635,PE89038,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-09,1989-08-29,89V140000,UNDERHOOD FIRES,===
45246,PE89050,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-17,1989-07-19,None,UNDERHOOD FIRES,===
...,...,...,...,...,...,...,...,...,...,...,...
281478,PE89102,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-03-15,1989-08-01,None,HEATER HOSE ROUTING,===
298344,PE89045,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-17,1989-08-29,89V142000,UNDERHOOD FIRES,===
298349,PE89046,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-17,1989-05-05,None,UNDERHOOD FIRES,===
298357,PE89043,FORD,E350,1983,VISIBILITY:DEFROSTER/DEFOGGER/HVAC SYSTEM,Ford Motor Company,1989-02-15,1989-08-02,None,HEATER HOSE MISROUTING,===


In [45]:
#dropping all duplicates values in the primary keys. not performing any aggregation and imputation since we only need indicator for investigation
inv_unique = investigations.drop_duplicates(subset=['MAKE', 'MODEL', 'YEAR', 'COMPNAME'])

In [46]:
inv_unique.shape

(40805, 11)

In [ ]:
# Save DataFrame to CSV
# csv_filename_inv = r"C:\Users\shanm\OneDrive\Desktop\Kavya\Spring 25\NHTSA dataset-20250211T191427Z-002\NHTSA dataset\Investigations\investigations_unique.csv"
# df.to_csv(csv_filename_inv, index=False, encoding='utf-8')

In [ ]:
# csv_filename_recall = r"C:\Users\shanm\OneDrive\Desktop\Kavya\Spring 25\NHTSA dataset-20250211T191427Z-002\NHTSA dataset\Recall\recall_unique.csv"
 # df.to_csv(csv_filename_recall, index=False, encoding='utf-8')

In [47]:
# Filter for common 'MAKETXT' values
common_makes = set(inv_unique['MAKE']).intersection(RCL_unique['MAKETXT'].unique())
 
# Print common values
print("Common 'MAKETXT' values:", common_makes)
 
# Print status message
if common_makes:
    print("There are matching 'MAKETXT' values in both DataFrames.")
else:
    print("There are no matching 'MAKETXT' values in both DataFrames.")

Common 'MAKETXT' values: {'MONON', 'LAG', 'CYBEX', 'MICHELIN', 'MALLARD', 'AM GENERAL', 'STURDICORP', 'AEROLITE', 'ROCKWOOD', 'LOMAR', 'GRANVILLE', 'SEVAS', 'ATHEY', 'CARPENTER', 'BEALL', 'JOHN DEERE', 'Q STRAINT', 'NEWAY', 'JL TANK', 'MERCEDES BENZ', 'GRUMMAN', 'ALPHA ELITE', 'HYUNDAI TRANSLEAD', 'WAYNE', 'CHAMPION SIDECAR', 'SHASTA', 'TBII', 'HART', 'MONACO COACH', 'LINCOLN', 'ROAD RESCUE', 'NOVABUS', 'ELDORADO', 'ACC', 'VULCAN', 'SAFARI COACH', 'REYNOLDS', 'PONTIAC', 'GRACO', 'SPECIALTY MANUFACTURING', 'THOR', 'HAULMARK', 'COSCO', 'ROLLS-ROYCE', 'MACK', 'DUCATI', 'LIFT-U', 'OSHKOSH', 'MID BUS', 'GEO', 'CADILLAC', 'BEAVER', 'STEWART AND STEVENSON', 'CLEMENT', 'WORKHORSE', 'THOMAS BUILT BUSES', 'ELKHART', '4-STAR TRAILERS', 'THOR MOTOR', 'RENOLUX', 'GREAT DANE', 'NEW FLYER', 'PERRIS VALLEY', 'EMERGENCY ONE', 'K-TOOL', 'AUTOLIV', 'PILOT', 'LOTUS', 'TIFFIN', 'BRITAX', 'CENCORP', 'CLOUGH', 'KAWASAKI', 'TRAILSTAR', 'POLARIS', 'INDIAN', 'CHAMPION', 'LAFRANCE', 'QUALITY', 'JONWAY', 'SPARTAN

In [51]:
#making sure that all primary keys across three datasets are same for merging effectively
import re
 
# Standardize column to match investigation dataset
def clean_make_column(make):
    if pd.isna(make):  # Handle NaN values
        return None
    make = make.strip().upper()  # Remove spaces and convert to uppercase
    #make = re.sub(r'[^A-Z0-9 -]', '', make)  # Remove non-alphanumeric characters except spaces and hyphens
    make = re.sub(r'\s+', ' ', make)  # Normalize multiple spaces
 
    return make
 

In [52]:
# Apply cleaning function in investigation
inv_unique['MAKE'] = inv_unique['MAKE'].apply(clean_make_column)
inv_unique['MODEL'] = inv_unique['MODEL'].apply(clean_make_column)
inv_unique['YEAR'] = inv_unique['YEAR'].apply(clean_make_column)
inv_unique['COMPNAME'] = inv_unique['COMPNAME'].apply(clean_make_column)

In [57]:
# Apply cleaning function in recall
RCL_unique['MAKETXT'] = RCL_unique['MAKETXT'].apply(clean_make_column)
RCL_unique['MODELTXT'] = RCL_unique['MODELTXT'].apply(clean_make_column)
RCL_unique['YEARTXT'] = RCL_unique['YEARTXT'].apply(clean_make_column)
RCL_unique['COMPNAME'] = RCL_unique['COMPNAME'].apply(clean_make_column)

In [58]:
#adding additional column to investigation dataset for acting as indicator for investigation for the primary keys
inv_unique['INV_IND'] = 1

In [59]:
#adding additional column to recall dataset for acting as indicator for investigation for the primary keys
RCL_unique['RCL_IND'] = 1

In [62]:
#creating target variable and taking only relevant columns for the dataframe
target = inv_unique.merge(RCL_unique, how = 'left', left_on = ['MAKE', 'MODEL', 'YEAR', 'COMPNAME'], right_on = ['MAKETXT', 'MODELTXT', 'YEARTXT', 'COMPNAME'])[['MAKE', 'MODEL', 'YEAR', 'COMPNAME','INV_IND', 'RCL_IND']]

In [80]:
target['TARGET']= target['INV_IND'] + target['RCL_IND']

In [81]:
target.head()

,MAKE,MODEL,YEAR,COMPNAME,INV_IND,RCL_IND,TARGET
0,PACE AMERICAN,TRAILER,2003,WHEELS,1,0.0,1.0
1,PACE AMERICAN,TRAILER,2004,WHEELS,1,0.0,1.0
2,PACE AMERICAN,TRAILER,2005,WHEELS,1,0.0,1.0
3,PACE AMERICAN,TRAILER,2006,WHEELS,1,0.0,1.0
4,APEXCONE,APX9004,9999,EXTERIOR LIGHTING,1,1.0,2.0


In [79]:
import numpy as np
target['RCL_IND'] = np.where(target['RCL_IND'].isna(),0,target['RCL_IND'])

In [83]:
#Saving the dataframe to csv
csv_filename_target = r"C:\Users\shanm\OneDrive\Desktop\Kavya\Spring 25\NHTSA dataset-20250211T191427Z-002\NHTSA dataset\Target\target_only_rcl_inv.csv"
target.to_csv(csv_filename_target, index=False, encoding='utf-8')

In [82]:
#Note that there is no 0 which is low risk indicator present in this because this is formed by only adding recall and investigation dataset. 
#This target in merged with complaints dataset to get the final target
target['TARGET'].value_counts()

1.0    36268
2.0     4537
Name: TARGET, dtype: int64